In [2]:

import pandas as pd
import os
import json

from geopy.geocoders import Nominatim

In [55]:
# Using file from Main Reference website
parking_df = pd.read_csv('http://iot.ee.surrey.ac.uk:8080/datasets/parking/aarhus_parking.csv', sep=',')
parking_df.head()

,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime
0,0,2014-05-22 09:09:04.145,1,65,NORREPORT,2014-11-03 16:18:44
1,0,2014-05-22 09:09:04.145,2,512,SKOLEBAKKEN,2014-11-03 16:18:44
2,869,2014-05-22 09:09:04.145,3,1240,SCANDCENTER,2014-11-03 16:18:44
3,22,2014-05-22 09:09:04.145,4,953,BRUUNS,2014-11-03 16:18:44
4,124,2014-05-22 09:09:04.145,5,130,BUSGADEHUSET,2014-11-03 16:18:44


In [43]:
# update garage code to geolocation search to work
parking_df.loc[parking_df['garagecode'] == 'SCANDCENTER', 'garagecode'] = 'SCANDINAVIAN CENTER'
parking_df.head()

,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime
0,0,2014-05-22 09:09:04.145,1,65,NORREPORT,2014-11-03 16:18:44
1,0,2014-05-22 09:09:04.145,2,512,SKOLEBAKKEN,2014-11-03 16:18:44
2,869,2014-05-22 09:09:04.145,3,1240,SCANDINAVIAN CENTER,2014-11-03 16:18:44
3,22,2014-05-22 09:09:04.145,4,953,BRUUNS,2014-11-03 16:18:44
4,124,2014-05-22 09:09:04.145,5,130,BUSGADEHUSET,2014-11-03 16:18:44


In [51]:
# Process geolocation from garageCodes using GeoPy

 

garages = parking_df.groupby("garagecode")
print(garages.count())

garage_codes = garages['garagecode'].unique()
print(garage_codes)


# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")
 
# entering the location name
for row in garage_codes:
    print(f'{row[0]}, Aarhus, Denmark') 
    getLoc = loc.geocode(f'{row[0]}, Aarhus, Denmark')
    parking_df.loc[parking_df['garagecode'] == row[0], 'latitude'] = getLoc.latitude
    parking_df.loc[parking_df['garagecode'] == row[0],'longitude'] = getLoc.longitude
    parking_df.loc[parking_df['garagecode'] == row[0],'address'] = getLoc.address
    

                     vehiclecount  updatetime   _id  totalspaces  streamtime
garagecode                                                                  
BRUUNS                       6908        6908  6908         6908        6908
BUSGADEHUSET                 6908        6908  6908         6908        6908
KALKVAERKSVEJ                6908        6908  6908         6908        6908
MAGASIN                      6908        6908  6908         6908        6908
NORREPORT                    6908        6908  6908         6908        6908
SALLING                      6908        6908  6908         6908        6908
SCANDINAVIAN CENTER          6908        6908  6908         6908        6908
SKOLEBAKKEN                  6908        6908  6908         6908        6908
garagecode
BRUUNS                              [BRUUNS]
BUSGADEHUSET                  [BUSGADEHUSET]
KALKVAERKSVEJ                [KALKVAERKSVEJ]
MAGASIN                            [MAGASIN]
NORREPORT                        [NORRE

In [52]:
parking_df.head()

,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime,latitude,longitude,address
0,0,2014-05-22 09:09:04.145,1,65,NORREPORT,2014-11-03 16:18:44,56.162023,10.211065,"Nørreport, Aarhus, Aarhus Kommune, Region Midt..."
1,0,2014-05-22 09:09:04.145,2,512,SKOLEBAKKEN,2014-11-03 16:18:44,56.157146,10.213121,"Skolebakken, Aarhus, Aarhus Kommune, Region Mi..."
2,869,2014-05-22 09:09:04.145,3,1240,SCANDINAVIAN CENTER,2014-11-03 16:18:44,56.152770,10.197070,"Scandinavian Center Aarhus, Skovgaardsgade, Fr..."
3,22,2014-05-22 09:09:04.145,4,953,BRUUNS,2014-11-03 16:18:44,56.149019,10.204733,"Bruuns Galleri, Værkmestergade, Midtbyen, Fred..."
4,124,2014-05-22 09:09:04.145,5,130,BUSGADEHUSET,2014-11-03 16:18:44,56.155552,10.206232,"Busgadehuset, Busgaden, Midtbyen, Frederiksbje..."


In [54]:
# This is the file we will use in our analyze 
parking_df.to_csv('../data/aarhus_parking_geolocated.csv')

## Georeferencing


In [14]:
import folium

AARHUS_GEOLOCATION = [56.16182966470494, 10.197421860666816]

parking_df = pd.read_csv('../data/aarhus_parking_geolocated.csv', sep=',')
map = folium.Map(location=AARHUS_GEOLOCATION, zoom_start=13)
garage_codes = parking_df.groupby("garagecode")['address'].unique()

garage_codes


garagecode
BRUUNS                 [Bruuns Galleri, Værkmestergade, Midtbyen, Fre...
BUSGADEHUSET           [Busgadehuset, Busgaden, Midtbyen, Frederiksbj...
KALKVAERKSVEJ          [Kalkværksvej, Sydhavnskvarteret, Frederiksbje...
MAGASIN                [Magasin, Åboulevarden, Midtbyen, Aarhus, Aarh...
NORREPORT              [Nørreport, Aarhus, Aarhus Kommune, Region Mid...
SALLING                [Salling, Ferdinand Sallings Stræde, Midtbyen,...
SCANDINAVIAN CENTER    [Scandinavian Center Aarhus, Skovgaardsgade, F...
SKOLEBAKKEN            [Skolebakken, Aarhus, Aarhus Kommune, Region M...
Name: address, dtype: object

In [10]:

map